# Masterthesis
## Regression

#### Julian Jetz

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import category_encoders as ce
from tensorflow.python.data import Dataset
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import Input, Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor 
from sklearn.model_selection import RandomizedSearchCV, KFold 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler, LabelBinarizer, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
import matplotlib.pyplot as plt
from scipy import stats

Using TensorFlow backend.


In [2]:
na_values = ['nan', 'N/A', 'NaN', 'NaT']

In [3]:
df = pd.read_csv('resources/accident_data.csv', sep=";", na_values=na_values, index_col=False, dtype = {"STATE" : "str", "TYPE" : "str", "TYPEQ" : "str", "WEATHER" : "str", "VISIBLTY" : "str", "SPD_TOO_HIGH" : "str"})

In [4]:
df=df.dropna(subset=['WEATHER', 'VISIBLTY'])
pd.set_option('display.max_columns', 500)
#df.head(50)

## Vorbereitungen

Festlegen der abhängigen Variablen X und der vorherzusagenden Variable y

In [5]:
x_data = df[['TEMP', 'TYPEQ', 'TRKCLAS', 'ACCCAUSE_LVL1', 'ACCCAUSE_LVL2', 'ACCAUSE', 'YEAR4', 'STATE', 'MONTH', 'LOCOMOTIVES1', 'LOADF1', 'LOADP1', 'EMPTYF1', 'EMPTYP1','LOCOMOTIVES2', 'LOADF2', 'EMPTYF2', 'LOADP2', 'EMPTYP2', 'TONS', 'TEMP_CLUSTER', 'SPEED', 'ACCCAUSE_LVL1', 'TYPE', 'EQUIPMENT_TYPE', 'TYPTRK', 'VISIBLTY', 'WEATHER', 'SPD_TOO_HIGH', 'ACCTYPE', 'ACCIDENT_TYPE', 'Jahreszeit']]

In [6]:
y = df['INFRASTRUCTURE_DMG']

In [7]:
print(x_data.isnull().sum())

TEMP              0
TYPEQ             0
TRKCLAS           0
ACCCAUSE_LVL1     0
ACCCAUSE_LVL2     0
ACCAUSE           0
YEAR4             0
STATE             0
MONTH             0
LOCOMOTIVES1      0
LOADF1            0
LOADP1            0
EMPTYF1           0
EMPTYP1           0
LOCOMOTIVES2      0
LOADF2            0
EMPTYF2           0
LOADP2            0
EMPTYP2           0
TONS              0
TEMP_CLUSTER      0
SPEED             0
ACCCAUSE_LVL1     0
TYPE              0
EQUIPMENT_TYPE    0
TYPTRK            0
VISIBLTY          0
WEATHER           0
SPD_TOO_HIGH      0
ACCTYPE           0
ACCIDENT_TYPE     0
Jahreszeit        0
dtype: int64


## Umwandlung der kategorialen Merkmale und Normalisierung der kontinuierlichen Merkmale

In [8]:
# Funktion zur Umwandlung der kategorialen Merkmale mit dem One Hot Encoder 
# x = kategoriale Merkmale
def prepare_inputs_One_Hot(x):
    encoder = OneHotEncoder(sparse=False)
    encoder.fit(x)
    x_enc = encoder.transform(x)
    return x_enc

In [9]:
# Funktion zur Umwandlung der kategorialen Merkmale mit dem Ordinal Encoder 
# x = kategoriale Merkmale
def prepare_inputs_Ordinal(x):
    encoder = OrdinalEncoder()
    encoder.fit(x)
    x_enc = encoder.transform(x)
    return x_enc

In [10]:
# Funktion zur Umwandlung der kategorialen Merkmale mit dem Binary Encoder
# x = Datensatz, cats = Liste mit Spaltenüberschriften der kategorialen Merkmale
def prepare_inputs_Binary(x, cats):
    encoder = ce.BinaryEncoder(cols=cats, return_df=True)
    encoder.fit(x[cats])
    df_bin = encoder.transform(x[cats])
    return df_bin

In [11]:
# Funktion zur Umwandlung der kategorialen Merkmale mit dem Target Encoder
# x = Datensatz, cats = Liste mit Spaltenüberschriften der kategorialen Merkmale, Y = unabhägiges Merkmal
def prepare_inputs_Target(x, Y, cats, smoothing=1):
    encoder = ce.TargetEncoder(cols=cats, return_df=True, min_samples_leaf = 5, smoothing=smoothing)
    encoder.fit(x[cats], Y)
    df_te = encoder.transform(x[cats], Y)
    return df_te

In [12]:
# Funktion zur Umwandlung der kategorialen Merkmale mit dem Leave One Out Encoder
# x = Datensatz, cats = Liste mit Spaltenüberschriften der kategorialen Merkmale, Y = unabhägiges Merkmal
def prepare_inputs_Leave_One_Out(x, Y, cats):
    encoder = ce.LeaveOneOutEncoder(cols=cats, return_df=True)
    encoder.fit(x[cats], Y)
    df_loo = encoder.transform(x[cats], Y)
    return df_loo

### Merkmale, die betrachtet werden





In [13]:
#cat = ['TEMP_CLUSTER', 'Jahreszeit', 'EQUIPMENT_TYPE', 'ACCIDENT_TYPE', 'TYPTRK', 'WEATHER', 'VISIBLTY', 'STATE', 'ACCCAUSE_LVL1', 'SPD_TOO_HIGH', 'ACCTYPE', 'TRKCLAS']

In [14]:
#con = ['SPEED', 'TONS', 'YEAR4', 'LOADF1', 'EMPTYF1', 'LOADP1', 'EMPTYP1', 'LOCOMOTIVES1', 'LOADF2', 'EMPTYF2', 'LOADP2', 'EMPTYP2', 'LOCOMOTIVES2']

In [15]:
cat = ['TEMP_CLUSTER', 'Jahreszeit', 'EQUIPMENT_TYPE', 'ACCIDENT_TYPE', 'ACCTYPE', 'TRKCLAS']

In [16]:
con = ['SPEED', 'TONS', 'YEAR4', 'LOADF1', 'EMPTYF1', 'LOADP1', 'EMPTYP1', 'LOCOMOTIVES1' , 'LOADF2', 'EMPTYF2', 'LOADP2', 'EMPTYP2', 'LOCOMOTIVES2']

In [17]:
#x_enc_cat = prepare_inputs_Ordinal(x_data[cat])

In [18]:
x_enc_cat = prepare_inputs_One_Hot(x_data[cat])

In [19]:
#x_enc_cat = prepare_inputs_Binary(x_data, cat)

In [20]:
#x_enc_cat = prepare_inputs_Target(x_data, y, cat)

In [21]:
#x_enc_cat = prepare_inputs_Leave_One_Out(x_data, y, cat)

In [22]:
# Normalisierung der kontinuierlichen Merkmale mit dem StandardScaler oder dem MinMaxScaler
#cs = MinMaxScaler().fit(x_data[con])
cs = StandardScaler().fit(x_data[con])
x_enc_con = cs.transform(x_data[con])
#x_enc_con = x_data[con]

In [23]:
#cs_y = MinMaxScaler().fit(y.values.reshape(-1, 1))
#cs_y = StandardScaler().fit(y.values.reshape(-1, 1))
#y = cs_y.transform(y.values.reshape(-1, 1))

#### Zusammenfügen der kategorialen und kontinuierlichen Merkmale

In [24]:
X_Data = np.hstack([x_enc_cat, x_enc_con])

#### Aufteilen der Merkmale in Trainings- und Testdaten
70% Trainingsdaten & 30 % Testdaten
random_state: Kontrolliert die Zufallsverteilung -> Es werden immer die gleichen Werte Trainings- und Testdaten zugeteilt. Sonst könnte es zu Abweichungen bei den Ergebnissen des Modells kommen

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X_Data, y, test_size=0.3, random_state=42)

In [26]:
n_features = X_train.shape[1]

## Sequential regression model

In [27]:
def create_model(learning_rate = 0.001, activation = 'relu'):
    seq_model = Sequential()
    dropout = 0
    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
     
    seq_model.add(Dense(512, activation=activation, input_shape=(n_features,)))
    #seq_model.add(BatchNormalization())
    seq_model.add(Dropout(dropout))
    seq_model.add(Dense(256, activation=activation))
    #seq_model.add(Dense(128, activation=activation))
    #seq_model.add(Dropout(dropout))
    #seq_model.add(Dense(64, activation=activation))
    #seq_model.add(Dropout(dropout))
    #seq_model.add(Dense(32, activation=activation))
    #seq_model.add(Dropout(dropout))
    seq_model.add(Dense(1))
    seq_model.compile(optimizer=opt, loss='mse', metrics=['mse', tf.keras.metrics.RootMeanSquaredError(name='rmse'), 'mae'])
    return seq_model

In [28]:
model = KerasRegressor(build_fn = create_model)

In [29]:
params = {'activation': ["relu", "tanh", "sigmoid", "selu", "softmax"],'epochs': [5, 50, 100, 300, 500, 1000],'learning_rate': [0.001, 0.009, 0.008, 0.0007, 0.0006, 0.0001]}

In [30]:
random_search = RandomizedSearchCV(model, param_distributions = params,cv = KFold(10))

In [ ]:
random_search_results = random_search.fit(X_train, y_train)

Epoch 1/50
1246/1246 [==============================] - 2s 1ms/step - loss: 18639376384.0000 - mse: 18639376384.0000 - rmse: 136526.0938 - mae: 59932.9297
Epoch 2/50
1246/1246 [==============================] - 2s 1ms/step - loss: 9677044736.0000 - mse: 9677044736.0000 - rmse: 98371.9688 - mae: 48840.1328
Epoch 3/50
1246/1246 [==============================] - 2s 1ms/step - loss: 7998986240.0000 - mse: 7998986240.0000 - rmse: 89437.0547 - mae: 45813.6289
Epoch 4/50
1246/1246 [==============================] - 2s 2ms/step - loss: 7687219200.0000 - mse: 7687219200.0000 - rmse: 87676.7891 - mae: 45063.2656
Epoch 5/50
1246/1246 [==============================] - 2s 1ms/step - loss: 7358686208.0000 - mse: 7358686208.0000 - rmse: 85782.7812 - mae: 42057.0430
Epoch 6/50
1246/1246 [==============================] - 2s 1ms/step - loss: 7047597056.0000 - mse: 7047597056.0000 - rmse: 83949.9688 - mae: 39994.0273
Epoch 7/50
1246/1246 [==============================] - 2s 1ms/step - loss: 685503641

1246/1246 [==============================] - 2s 1ms/step - loss: 8047272960.0000 - mse: 8047272960.0000 - rmse: 89706.5938 - mae: 46106.1992
Epoch 4/50
1246/1246 [==============================] - 2s 1ms/step - loss: 7710795776.0000 - mse: 7710795776.0000 - rmse: 87811.1406 - mae: 44447.8438
Epoch 5/50
1246/1246 [==============================] - 2s 1ms/step - loss: 7315530240.0000 - mse: 7315530240.0000 - rmse: 85530.8750 - mae: 41104.1172
Epoch 6/50
1246/1246 [==============================] - 2s 1ms/step - loss: 7024451584.0000 - mse: 7024451584.0000 - rmse: 83812.0000 - mae: 39235.7891
Epoch 7/50
1246/1246 [==============================] - 2s 1ms/step - loss: 6856780288.0000 - mse: 6856780288.0000 - rmse: 82805.6797 - mae: 38626.5625
Epoch 8/50
1246/1246 [==============================] - 2s 1ms/step - loss: 6760600064.0000 - mse: 6760600064.0000 - rmse: 82222.8672 - mae: 38277.8828
Epoch 9/50
1246/1246 [==============================] - 2s 1ms/step - loss: 6687842304.0000 - mse: 

1246/1246 [==============================] - 2s 1ms/step - loss: 7238002176.0000 - mse: 7238002176.0000 - rmse: 85076.4453 - mae: 41344.4961
Epoch 6/50
1246/1246 [==============================] - 2s 1ms/step - loss: 6958428160.0000 - mse: 6958428160.0000 - rmse: 83417.1953 - mae: 39381.7500
Epoch 7/50
1246/1246 [==============================] - 2s 1ms/step - loss: 6792983040.0000 - mse: 6792983040.0000 - rmse: 82419.5547 - mae: 38600.0977
Epoch 8/50
1246/1246 [==============================] - 2s 1ms/step - loss: 6695546368.0000 - mse: 6695546368.0000 - rmse: 81826.3203 - mae: 38256.9922
Epoch 9/50
1246/1246 [==============================] - 2s 1ms/step - loss: 6629013504.0000 - mse: 6629013504.0000 - rmse: 81418.7578 - mae: 38039.9688
Epoch 10/50
1246/1246 [==============================] - 2s 1ms/step - loss: 6585890304.0000 - mse: 6585890304.0000 - rmse: 81153.5000 - mae: 38082.0039
Epoch 11/50
1246/1246 [==============================] - 2s 1ms/step - loss: 6563751936.0000 - mse

1246/1246 [==============================] - 2s 1ms/step - loss: 6742326272.0000 - mse: 6742326272.0000 - rmse: 82111.6719 - mae: 38503.6875
Epoch 8/50
1246/1246 [==============================] - 2s 1ms/step - loss: 6636982272.0000 - mse: 6636982272.0000 - rmse: 81467.6797 - mae: 38178.6992
Epoch 9/50
1246/1246 [==============================] - 2s 1ms/step - loss: 6570133504.0000 - mse: 6570133504.0000 - rmse: 81056.3594 - mae: 38056.0078
Epoch 10/50
1246/1246 [==============================] - 2s 1ms/step - loss: 6523406848.0000 - mse: 6523406848.0000 - rmse: 80767.6094 - mae: 37904.6758
Epoch 11/50
1246/1246 [==============================] - 2s 1ms/step - loss: 6501613056.0000 - mse: 6501613056.0000 - rmse: 80632.5781 - mae: 37987.6641
Epoch 12/50
1246/1246 [==============================] - 2s 1ms/step - loss: 6479564288.0000 - mse: 6479564288.0000 - rmse: 80495.7422 - mae: 37918.6133
Epoch 13/50
1246/1246 [==============================] - 2s 1ms/step - loss: 6460856320.0000 - m

1246/1246 [==============================] - 2s 1ms/step - loss: 6722614272.0000 - mse: 6722614272.0000 - rmse: 81991.5469 - mae: 38226.2656
Epoch 10/50
1246/1246 [==============================] - 2s 1ms/step - loss: 6680951296.0000 - mse: 6680951296.0000 - rmse: 81737.0859 - mae: 38114.2344
Epoch 11/50
1246/1246 [==============================] - 2s 1ms/step - loss: 6649939456.0000 - mse: 6649939456.0000 - rmse: 81547.1641 - mae: 38117.7109
Epoch 12/50
1246/1246 [==============================] - 2s 1ms/step - loss: 6630269440.0000 - mse: 6630269440.0000 - rmse: 81426.4688 - mae: 38040.7305
Epoch 13/50
1246/1246 [==============================] - 2s 1ms/step - loss: 6615388160.0000 - mse: 6615388160.0000 - rmse: 81335.0391 - mae: 38039.7617
Epoch 14/50
1246/1246 [==============================] - 2s 1ms/step - loss: 6596048896.0000 - mse: 6596048896.0000 - rmse: 81216.0625 - mae: 37967.4023
Epoch 15/50
1246/1246 [==============================] - 2s 1ms/step - loss: 6583139840.0000 -

...

1246/1246 [==============================] - 2s 2ms/step - loss: 24377575424.0000 - mse: 24377575424.0000 - rmse: 156133.2031 - mae: 69947.0703
Epoch 37/300
1246/1246 [==============================] - 2s 1ms/step - loss: 24377354240.0000 - mse: 24377354240.0000 - rmse: 156132.4844 - mae: 69945.7734
Epoch 38/300
1246/1246 [==============================] - 2s 1ms/step - loss: 24377190400.0000 - mse: 24377190400.0000 - rmse: 156131.9688 - mae: 69944.3984
Epoch 39/300
1246/1246 [==============================] - 2s 1ms/step - loss: 24376967168.0000 - mse: 24376967168.0000 - rmse: 156131.2500 - mae: 69942.8594
Epoch 40/300
1246/1246 [==============================] - 2s 1ms/step - loss: 24376768512.0000 - mse: 24376768512.0000 - rmse: 156130.6094 - mae: 69941.5234
Epoch 41/300
1246/1246 [==============================] - 2s 1ms/step - loss: 24376559616.0000 - mse: 24376559616.0000 - rmse: 156129.9375 - mae: 69940.1328
Epoch 42/300
1246/1246 [==============================] - 2s 1ms/step -

1246/1246 [==============================] - 2s 2ms/step - loss: 24367521792.0000 - mse: 24367521792.0000 - rmse: 156101.0000 - mae: 69875.2969
Epoch 88/300
1246/1246 [==============================] - 2s 2ms/step - loss: 24367306752.0000 - mse: 24367306752.0000 - rmse: 156100.3125 - mae: 69873.8359
Epoch 89/300
1246/1246 [==============================] - 2s 2ms/step - loss: 24367110144.0000 - mse: 24367110144.0000 - rmse: 156099.6875 - mae: 69872.4297
Epoch 90/300
1246/1246 [==============================] - 2s 2ms/step - loss: 24366950400.0000 - mse: 24366950400.0000 - rmse: 156099.1719 - mae: 69871.0234
Epoch 91/300
1246/1246 [==============================] - 2s 2ms/step - loss: 24366743552.0000 - mse: 24366743552.0000 - rmse: 156098.5000 - mae: 69869.6094
Epoch 92/300
1246/1246 [==============================] - 2s 2ms/step - loss: 24366526464.0000 - mse: 24366526464.0000 - rmse: 156097.8125 - mae: 69868.2188
Epoch 93/300
1246/1246 [==============================] - 2s 2ms/step -

1246/1246 [==============================] - 2s 2ms/step - loss: 24357287936.0000 - mse: 24357287936.0000 - rmse: 156068.2188 - mae: 69801.9688
Epoch 140/300
1246/1246 [==============================] - 2s 2ms/step - loss: 24357083136.0000 - mse: 24357083136.0000 - rmse: 156067.5625 - mae: 69800.5703
Epoch 141/300
1246/1246 [==============================] - 2s 2ms/step - loss: 24356902912.0000 - mse: 24356902912.0000 - rmse: 156066.9844 - mae: 69799.1875
Epoch 142/300
1246/1246 [==============================] - 2s 2ms/step - loss: 24356714496.0000 - mse: 24356714496.0000 - rmse: 156066.3750 - mae: 69797.7422
Epoch 143/300
1246/1246 [==============================] - 2s 2ms/step - loss: 24356499456.0000 - mse: 24356499456.0000 - rmse: 156065.6875 - mae: 69796.3125
Epoch 144/300
1246/1246 [==============================] - 2s 2ms/step - loss: 24356308992.0000 - mse: 24356308992.0000 - rmse: 156065.0781 - mae: 69795.0078A: 0s - loss: 24023734272.0000 - mse: 24023734272.0000 - rmse: 1549

1246/1246 [==============================] - 2s 1ms/step - loss: 24347453440.0000 - mse: 24347453440.0000 - rmse: 156036.7031 - mae: 69731.5859
Epoch 190/300
1246/1246 [==============================] - 2s 1ms/step - loss: 24347260928.0000 - mse: 24347260928.0000 - rmse: 156036.0938 - mae: 69730.1406
Epoch 191/300
1246/1246 [==============================] - 2s 1ms/step - loss: 24347076608.0000 - mse: 24347076608.0000 - rmse: 156035.5000 - mae: 69728.7344
Epoch 192/300
1246/1246 [==============================] - 2s 1ms/step - loss: 24346847232.0000 - mse: 24346847232.0000 - rmse: 156034.7656 - mae: 69727.3359
Epoch 193/300
1246/1246 [==============================] - 2s 1ms/step - loss: 24346669056.0000 - mse: 24346669056.0000 - rmse: 156034.1875 - mae: 69725.9219
Epoch 194/300
1246/1246 [==============================] - 2s 1ms/step - loss: 24346472448.0000 - mse: 24346472448.0000 - rmse: 156033.5625 - mae: 69724.5234
Epoch 195/300
1246/1246 [==============================] - 2s 1ms/

1246/1246 [==============================] - 2s 1ms/step - loss: 24337438720.0000 - mse: 24337438720.0000 - rmse: 156004.6094 - mae: 69659.6406
Epoch 241/300
1246/1246 [==============================] - 2s 1ms/step - loss: 24337250304.0000 - mse: 24337250304.0000 - rmse: 156004.0000 - mae: 69658.2891
Epoch 242/300
1246/1246 [==============================] - 2s 1ms/step - loss: 24337053696.0000 - mse: 24337053696.0000 - rmse: 156003.3750 - mae: 69656.8828
Epoch 243/300
1246/1246 [==============================] - 2s 1ms/step - loss: 24336855040.0000 - mse: 24336855040.0000 - rmse: 156002.7344 - mae: 69655.4609
Epoch 244/300
1246/1246 [==============================] - 2s 1ms/step - loss: 24336646144.0000 - mse: 24336646144.0000 - rmse: 156002.0781 - mae: 69654.0938
Epoch 245/300
1246/1246 [==============================] - 2s 1ms/step - loss: 24336437248.0000 - mse: 24336437248.0000 - rmse: 156001.4062 - mae: 69652.5938
Epoch 246/300
1246/1246 [==============================] - 2s 1ms/

In [ ]:
print("Best Score: ",random_search_results.best_score_,"and Best Params: ",random_search_results.best_params_)

In [ ]:
def GridSearch_table_plot(grid_clf, param_name,
                          num_results=20,
                          negative=True,
                          graph=True,
                          display_all_params=True):

    '''Display grid search results

    Arguments
    ---------

    grid_clf           the estimator resulting from a grid search
                       for example: grid_clf = GridSearchCV( ...

    param_name         a string with the name of the parameter being tested

    num_results        an integer indicating the number of results to display
                       Default: 15

    negative           boolean: should the sign of the score be reversed?
                       scoring = 'neg_log_loss', for instance
                       Default: True

    graph              boolean: should a graph be produced?
                       non-numeric parameters (True/False, None) don't graph well
                       Default: True

    display_all_params boolean: should we print out all of the parameters, not just the ones searched for?
                       Default: True

    Usage
    -----

    GridSearch_table_plot(grid_clf, "min_samples_leaf")

                          '''
    from IPython.display import display

    clf = grid_clf.best_estimator_
    clf_params = grid_clf.best_params_
    if negative:
        clf_score = -grid_clf.best_score_
    else:
        clf_score = grid_clf.best_score_
    clf_stdev = grid_clf.cv_results_['std_test_score'][grid_clf.best_index_]
    cv_results = grid_clf.cv_results_

    print("best parameters: {}".format(clf_params))
    print("best score:      {:0.5f} (+/-{:0.5f})".format(clf_score, clf_stdev))
    if display_all_params:
        import pprint
        pprint.pprint(clf.get_params())

    # pick out the best results
    # =========================
    scores_df = pd.DataFrame(cv_results).sort_values(by='rank_test_score')

    best_row = scores_df.iloc[0, :]
    if negative:
        best_mean = -best_row['mean_test_score']
    else:
        best_mean = best_row['mean_test_score']
    best_stdev = best_row['std_test_score']
    best_param = best_row['param_' + param_name]

    # display the top 'num_results' results
    # =====================================
    display(pd.DataFrame(cv_results) \
            .sort_values(by='rank_test_score').head(num_results))

    # plot the results
    # ================
    scores_df = scores_df.sort_values(by='param_' + param_name)

    if negative:
        means = -scores_df['mean_test_score']
    else:
        means = scores_df['mean_test_score']
    stds = scores_df['std_test_score']
    params = scores_df['param_' + param_name]

    # plot
    if graph:
        plt.figure(figsize=(8, 8))
        plt.errorbar(params, means, yerr=stds)

        plt.axhline(y=best_mean + best_stdev, color='red')
        plt.axhline(y=best_mean - best_stdev, color='red')
        plt.plot(best_param, best_mean, 'or')

        plt.title(param_name + " vs Score\nBest Score {:0.5f}".format(clf_score))
        plt.xlabel(param_name)
        plt.ylabel('Score')
        plt.show()

In [ ]:
GridSearch_table_plot(random_search, "INF_DMG", negative=False)

### Regression Allgemein

In [ ]:
epochs = 150

In [ ]:
epochs

In [ ]:
learning_rate = 0.0008

In [ ]:
learning_rate

In [ ]:
activation = 'relu'

In [ ]:
activation

In [ ]:
dropout = 0

In [ ]:
dropout

In [ ]:
metrics = ['mse', tf.keras.metrics.RootMeanSquaredError(name='rmse'), 'mae']

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
#opt = tf.keras.optimizers.Adamax()
#opt = tf.keras.optimizers.Adagrad(learning_rate=learning_rate)
#opt = tf.keras.optimizers.Ftrl(learning_rate=learning_rate)
#opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
#opt = tf.keras.optimizers.Adadelta(learning_rate=learning_rate)

## Sequential Regression with Keras

In [ ]:
seq_model = Sequential()

#seq_model.add(Dense(int(n_features*2/3), activation=activation, input_shape=(n_features,)))
seq_model.add(Dense(512, activation=activation, input_shape=(n_features,)))
#seq_model.add(BatchNormalization())
seq_model.add(Dropout(dropout))
seq_model.add(Dense(256, activation=activation))
#seq_model.add(BatchNormalization())
#seq_model.add(Dense(int(n_features*2/3), activation=activation))
#seq_model.add(Dropout(dropout))
#seq_model.add(Dense(64, activation=activation))
#seq_model.add(Dropout(dropout))
#seq_model.add(Dense(32, activation=activation))
#seq_model.add(Dropout(dropout))
seq_model.add(Dense(1))
seq_model.compile(optimizer=opt, loss='mse', metrics=metrics)

In [ ]:
seq_model.summary()

In [ ]:
plot_model(seq_model, 'Images/Sequential Regression Model.png', show_shapes=True)

In [ ]:
es = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
history = seq_model.fit(X_train, y_train, validation_data=(X_test, y_test), 
                        epochs=epochs, batch_size=32)

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.title('Learning Curves')
plt.xlabel('Epoch')
plt.ylabel('Cross Entropy')
plt.legend()
plt.show()

In [ ]:
# Evaluation des Regressionsmodells
error = seq_model.evaluate(X_test, y_test)

In [ ]:
print('MSE: %.3f, RMSE: %.3f, MAE: %.3f' % 
      (error[1], error[2], error[3]))

In [ ]:
# Scatter Plot mit den tatsächlichen und den vorhergesagten Daten für die Test Data

predictions= seq_model.predict(X_test)
plt.plot(y_test, predictions, 'o')
plt.xlabel('Tatsächliche Werte (Test Daten)')
plt.xticks(rotation=90)
plt.ylabel('Vorhergesagte Werte (Test Daten)')

In [ ]:
# Berechnung des R2-Scores (Bestimmtheitsmaß)
r2_seq = r2_score(y_test, predictions)
r2_seq

In [ ]:
# Berechnung des adjustierten R2-Scores (adj. Bestimmtheitsmaß)
adj_r2_seq = 1-(1-r2_seq)*((len(X_test)-1)/(len(X_test)-(len(cat)+len(con))))

In [ ]:
print(('%.6f \t %.6f \t %.0f  \t %.0f \t %.0f' % 
      (r2_seq, adj_r2_seq, error[2], error[1], error[3])).replace('.', ','))

In [ ]:
# Scatter Plot mit den tatsächlichen und den vorhergesagten Daten für die Training Data
train_predictions=seq_model.predict(X_train)
plt.plot(y_train, train_predictions, 'o')
plt.xlabel('Tatsächliche Werte (Trainings Daten)')
plt.xticks(rotation=90)
plt.ylabel('Vorhergesagte Werte (Trainings Daten)')

In [ ]:
predictions = predictions.flatten()

In [ ]:
train_predictions = train_predictions.flatten()

In [ ]:
pred = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})
pred1 = pred.sample(50)

pred1.plot(kind='bar',figsize=(20,16))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

In [ ]:
# Differenzen zwischen vorhergesagten und tatsächlichen Werten
plt.scatter(train_predictions, train_predictions - y_train, c='b', s=40, alpha=0.5, label='Trainings Daten')
plt.scatter(predictions, predictions - y_test, c='g', s=40, label='Test Daten')
plt.hlines(y=0, xmin=-0.03, xmax=0.2)
plt.title('Residuum Diagramm der Regression')
plt.ylabel('Residuum')
plt.xlabel('Infrastrukturschaden')
plt.legend()
plt.show()

## Functional API Regression Model with Keras

In [ ]:
x_in = Input(shape=(n_features,))
dense = Dense(1024, activation=activation)
x = dense(x_in)
x = Dropout(dropout)(x)
x = Dense(512, activation=activation)(x)
#x = Dense(256, activation=activation)(x)
#x = Dropout(dropout)(x)
#x = Dense(128, activation=activation)(x)
#x = Dropout(dropout)(x)
#x = Dense(64, activation=activation)(x)
#x = Dropout(dropout)(x)
x_out = Dense(1)(x)
func_model = Model(inputs=x_in, outputs=x_out)

In [ ]:
func_model.summary()

In [ ]:
plot_model(func_model, 'Images/Functional API Regression Model.png', show_shapes=True)

In [ ]:
func_model.compile(optimizer=opt, loss='mse', metrics=metrics)

In [ ]:
es = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
func_history = func_model.fit(X_train, y_train, validation_data=(X_test, y_test), 
                        epochs=epochs, batch_size=32)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(func_history.history['loss'], label='train')
plt.plot(func_history.history['val_loss'], label='val')
plt.title('Learning Curves')
plt.xlabel('Epoch')
plt.ylabel('Cross Entropy')
plt.legend()
plt.show()

In [ ]:
# Evaluation des Regressionsmodells
error_func = func_model.evaluate(X_test, y_test, verbose=0)
print('MSE: %.3f, RMSE: %.3f, MAE: %.3f' % 
      (error_func[1], error_func[2], error_func[3]))

In [ ]:
# Scatter Plot mit den tatsächlichen und den vorhergesagten Daten für die Test Data
func_predictions=func_model.predict(X_test)
plt.plot(y_test, func_predictions, 'o')
plt.xlabel('Tatsächliche Werte (Test Daten)')
plt.xticks(rotation=90)
plt.ylabel('Vorhergesagte Werte (Test Daten)')

In [ ]:
# Berechnung des R2-Scores (Bestimmtheitsmaß)
r2_func = r2_score(y_test, func_predictions)
r2_func

In [ ]:
# Berechnung des adjustierten R2-Scores (adj. Bestimmtheitsmaß)
adj_r2_func = 1-(1-r2_func)*((len(X_test)-1)/(len(X_test)-(len(cat)+len(con))))
adj_r2_func

In [ ]:
print(('%.6f \t %.6f \t %.0f  \t %.0f \t %.0f' % 
      (r2_func, adj_r2_func, error_func[2], error_func[1], error_func[3])).replace('.', ','))

In [ ]:
# Scatter Plot mit den tatsächlichen und den vorhergesagten Daten für die Training Data
func_train_predictions=func_model.predict(X_train)
plt.plot(y_train, func_train_predictions, 'o')
plt.xlabel('Tatsächliche Werte (Trainings Daten)')
plt.xticks(rotation=90)
plt.ylabel('Vorhergesagte Werte (Trainings Daten)')

In [ ]:
func_predictions = func_predictions.flatten()

In [ ]:
func_train_predictions = func_train_predictions.flatten()

In [ ]:
pred = pd.DataFrame({'Actual': y_test, 'Predicted': func_predictions})
pred1 = pred.sample(50)

pred1.plot(kind='bar',figsize=(20,16))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

In [ ]:
# Differenzen zwischen vorhergesagten und tatsächlichen Werten
plt.scatter(func_train_predictions, func_train_predictions - y_train, c='b', s=40, alpha=0.5, label='Trainings Daten')
plt.scatter(func_predictions, func_predictions - y_test, c='g', s=40, label='Test Daten')
plt.hlines(y=0, xmin=-0.03, xmax=0.2)
plt.title('Residuum Diagramm der Regression')
plt.ylabel('Residuum')
plt.xlabel('Infrastrukturschaden')
plt.legend()
plt.show()

## Polynomial Regression with Scikit-Learn
* Polynomial Regression
* Polynomial Regression with Lasso => Lasso Regression schrumpft Koeffizienten um absoluten Wert -> ausselektieren.
* Polynomial Regression with Ridge (Ridge Regression or Tikhonov regularization) => Ridge Regression schrumpft Koeffizienten um relativen Wert.

In [ ]:
def poly_regression(X_train, X_test, Y_train, Y_test, reg):
    reg.fit(X_train, Y_train)
    y_pred = reg.predict(X_test)
    coef = reg.coef_
    intercept = reg.intercept_
    r2 = r2_score(Y_test, y_pred)
    MSA = mean_absolute_error(Y_test, y_pred) 
    MSE = mean_squared_error(Y_test, y_pred)
    RMSE = np.sqrt(MSE)
    MSLE = mean_squared_log_error(Y_test, y_pred)
    
    return [r2, MSA, MSE, RMSE, MSLE]

In [ ]:
def create_polynomial_regression_model(degree, reg):  
    poly_feat = PolynomialFeatures(degree=degree)
    X_train_poly = poly_feat.fit_transform(X_train)
    poly = reg[1].fit(X_train_poly, y_train)
    y_train_pred = poly.predict(X_train_poly)
    y_pred = poly.predict(poly_feat.fit_transform(X_test))
    r2 = r2_score(y_test, y_pred)
    adj_r2 = 1-(1-r2)*((len(X_test)-1)/(len(X_test)-(len(cat)+len(con))))
    MSA = mean_absolute_error(y_test, y_pred) 
    MSE = mean_squared_error(y_test, y_pred)
    RMSE = np.sqrt(MSE)
    #MSLE = mean_squared_log_error(y_test, y_pred)
    
    #print("The model performance with %s and a degree of %s" % (reg[0], degree))
    #print("R2 score of test set is %s" % (r2))
    #print("RMSE score of test set is %s" % (RMSE))
    #print("MSE score of test set is %s" % (MSE))
    #print("-------------------------------------------")
    #print(("%s \t %s \t %.6f \t %.0f \t %.4f" % (reg[0], degree, r2, RMSE, MSE)).replace('.', ','))
    print(("%.6f \t %.6f \t %.0f \t %.0f \t %.0f" % (r2, adj_r2, RMSE, MSE, MAE)).replace('.', ','))
    return [r2, adj_r2, MSA, MSE, RMSE, reg[1].coef_]

In [ ]:
#poly = PolynomialFeatures(n_features)

In [ ]:
degrees = [10, 15, 20]
poly_models = [["Lasso", Lasso()], ["ElasticNet", ElasticNet()]]

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#for mod in poly_models:
#    for deg in degrees:
#        poly_reg = create_polynomial_regression_model(deg, mod)

In [ ]:
#poly_reg = poly_regression(X_train_poly, Y_train, X_test_poly, Y_test, LinearRegression())

In [ ]:
#poly_lasso_reg = poly_regression(X_train_poly, Y_train, X_test_poly, Y_test, Lasso())

In [ ]:
#poly_ridge_reg = poly_regression(X_train_poly, Y_train, X_test_poly, Y_test, Ridge())